<a href="https://colab.research.google.com/github/rishil1108/RishilCompSci/blob/main/mindmatch_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install all dependencies
!pip install --upgrade openai textblob pandas matplotlib fpdf gradio


In [ ]:
# Cell 2: Import libraries and configure API
import openai
from textblob import TextBlob
import pandas as pd
from datetime import datetime
from fpdf import FPDF
import matplotlib.pyplot as plt
import gradio as gr
import getpass

# API setup
client = openai.OpenAI(api_key=getpass.getpass("Enter your OpenAI API key:"))


Enter your OpenAI API key:··········


In [ ]:
# Cell 3: Journal storage and system prompt
JOURNAL_FILE = "mindmatch_journal.csv"

try:
    journal_df = pd.read_csv(JOURNAL_FILE)
except:
    journal_df = pd.DataFrame(columns=["Timestamp", "Reflection", "Response", "Mood", "Affirmation"])

session_count = len(journal_df)

SYSTEM_PROMPT = """
You are MindMatch, an AI-powered sports psychologist for collegiate tennis athletes.
You respond to athlete journal reflections after practices or matches.
You use CBT, performance psychology, and mindfulness strategies.
If they’re feeling down, validate their emotion, help reframe the situation, and offer a breathing or reset routine.
If they’re confident, build their momentum.
End with a mental skill: a cue word, affirmation, visualization strategy, or reflection question.
"""


In [ ]:
# Cell 4: Helper functions for mood, affirmations, prompts, saving
def detect_emotion(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.4:
        return "Positive"
    elif polarity < -0.3:
        return "Negative"
    else:
        return "Neutral"

def get_mood_summary(emotion):
    return {
        "Positive": "😊 You sound proud or motivated. Let’s keep that momentum going!",
        "Negative": "😟 It sounds like you’re frustrated or down. Let’s reset and move forward.",
        "Neutral": "😐 You seem centered or reflective. Let’s find growth in this moment."
    }[emotion]

def get_affirmation(emotion):
    return {
        "Positive": "🔥 Affirmation: 'Confidence comes from preparation — and I’m always preparing.'",
        "Negative": "🌊 Affirmation: 'Even in losses, I rise stronger and smarter.'",
        "Neutral": "🌱 Affirmation: 'I am in control of my breath, my focus, and my mindset.'"
    }[emotion]

def save_to_journal(reflection, response, mood, affirmation):
    global journal_df
    new_entry = {
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Reflection": reflection,
        "Response": response,
        "Mood": mood,
        "Affirmation": affirmation
    }
    journal_df = pd.concat([journal_df, pd.DataFrame([new_entry])], ignore_index=True)
    journal_df.to_csv(JOURNAL_FILE, index=False)


In [ ]:
import re

# Helper to remove emojis and special Unicode characters
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Fixed PDF export
def export_journal_as_pdf():
    if journal_df.empty:
        return None

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="MindMatch Journal", ln=True, align="C")
    pdf.ln(10)

    for _, row in journal_df.iterrows():
        timestamp = remove_emojis(str(row['Timestamp']))
        reflection = remove_emojis(str(row['Reflection']))
        response = remove_emojis(str(row['Response']))
        mood = remove_emojis(str(row['Mood']))
        affirmation = remove_emojis(str(row['Affirmation']))

        pdf.multi_cell(0, 10, txt=f"Date: {timestamp}")
        pdf.multi_cell(0, 10, txt=f"Reflection: {reflection}")
        pdf.multi_cell(0, 10, txt=f"Response: {response}")
        pdf.multi_cell(0, 10, txt=f"Mood: {mood}")
        pdf.multi_cell(0, 10, txt=f"Affirmation: {affirmation}")
        pdf.ln(5)

    filepath = "MindMatch_Journal.pdf"
    pdf.output(filepath)
    return filepath



In [ ]:
# Cell 6: Plot mood trend
def plot_mood_progress():
    if journal_df.empty:
        print("No data to visualize.")
        return

    mood_map = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
    journal_df['MoodScore'] = journal_df['Mood'].map(mood_map)
    journal_df['Timestamp'] = pd.to_datetime(journal_df['Timestamp'])

    plt.figure(figsize=(10, 5))
    plt.plot(journal_df['Timestamp'], journal_df['MoodScore'], marker='o', linestyle='-')
    plt.title("🧠 Mood Progress Over Time")
    plt.ylabel("Mood Score (2=Positive, 1=Neutral, 0=Negative)")
    plt.xlabel("Date")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# Cell 7: Gradio Web App with download button
def gradio_handler(reflection):
    emotion = detect_emotion(reflection)
    mood_summary = get_mood_summary(emotion)
    affirmation = get_affirmation(emotion)

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": reflection}
            ],
            temperature=0.7,
            max_tokens=700
        )
        reply = response.choices[0].message.content.strip()
        save_to_journal(reflection, reply, emotion, affirmation)

        full_response = f"{mood_summary}\n\n{reply}\n\n{affirmation}"
        pdf_path = export_journal_as_pdf()

        return full_response, pdf_path

    except Exception as e:
        return f"⚠️ Error: {str(e)}", None


with gr.Blocks() as demo:
    gr.Markdown("## 🧠 MindMatch: Virtual Sports Psychologist for Tennis Athletes")
    gr.Markdown("Reflect on your performance and get mental training advice. Your entries are saved and exportable.")

    with gr.Row():
        input_box = gr.Textbox(label="🎾 Journal Reflection", placeholder="How was your practice today?")
    with gr.Row():
        output_box = gr.Textbox(label="🧠 MindMatch Response")
    with gr.Row():
        download_pdf = gr.File(label="📄 Download Your Journal as PDF", interactive=True)

    submit_btn = gr.Button("Submit Reflection")
    submit_btn.click(fn=gradio_handler, inputs=input_box, outputs=[output_box, download_pdf])

demo.launch(share=True, debug=True)




Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://20c8784744edfeae3e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
